In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [ ]:
import pytorch_lightning as pl

In [ ]:
import pandas as pd
import ipyplot
import torch
import pytorch_lightning as pl
from sklearn.metrics import f1_score
%matplotlib inline

In [ ]:
from src.model import ImageClassifier
from src.utils import ImagesDataset, evaluate_model
from src.transforms import transform
from src import params

# Load dataset

In [ ]:
train_df = pd.read_csv('train_labelled.csv')
train_df.head()

In [ ]:
train_image_paths = train_df.img_path.values
train_labels = train_df.label.values

In [ ]:
len(train_image_paths)

In [ ]:
ipyplot.plot_class_tabs(train_image_paths, train_labels)

# Prepare model

In [ ]:
train_dataset = ImagesDataset(train_image_paths, train_labels, transform=transform)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, 
                                           batch_size=params.BATCH_SIZE,
                                           num_workers=16,
                                           shuffle=False)

In [12]:
# Run macos_cert_fix.py if you are getting ssl errors
model = ImageClassifier(learning_rate=params.LEARNING_RATE)

In [13]:
trainer = pl.Trainer(
    max_epochs=params.MAX_EPOCHS,
    log_every_n_steps=-1, # Change to enable tensorboard logging
    #gpus=-1, # Uncomment to use GPU training
)
trainer.fit(model, train_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:101: UserWarning: you defined a validation_step but have no val_dataloader. Skipping val loop
  rank_zero_warn(f"you defined a {step_name} but have no {loader_name}. Skipping {stage} loop")

  | Name  | Type         | Params
---------------------------------------
0 | trunk | EfficientNet | 2.2 M 
---------------------------------------
2.2 M     Trainable params
0         Non-trainable params
2.2 M     Total params
8.906     Total estimated model params size (MB)


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:322: UserWarning: The number of training samples (16) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Epoch 0:   0%|          | 0/16 [00:00<00:00, 394.13it/s]  

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1046: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


# Compute training set accuracy

In [14]:
labels, predictions = evaluate_model(model, train_dataset)

100%|██████████| 32/32 [00:32<00:00,  1.02s/it]


In [15]:
f1_score(labels, predictions, labels=1, average='binary')

0.14939309056956118

# Save model

In [16]:
trainer.save_checkpoint("model.ckpt")